<a href="https://colab.research.google.com/github/pujitha-vasanth/aspects_of_3d_image_reconstruction/blob/main/2D_to_3D_image_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Depth Map

In [3]:
# Import relevant libraries
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [4]:
from google.colab import files
uploaded = files.upload()

Saving cactus.jpg to cactus (1).jpg


In [5]:
input_image = plt.imread('cactus.jpg')
input_image = np.expand_dims(input_image, axis=0)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=input_image.shape[1:]),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(1, 3, activation='relu', padding='same')
])

In [7]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(input_image, input_image, epochs=10)

Epoch 1/10


In [ ]:
depth_map = model.predict(input_image)

In [ ]:
plt.imshow(depth_map[0, :, :, 0])
plt.show()

#Surface Plot

In [ ]:
import cv2

In [ ]:
# Load the input image
img= cv2.imread('teeth.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)



In [ ]:
# Compute the image gradient using Sobel operator (a derivative mask used for edge detection.) 
# It works by convolving the image with two small filters, 
# one for detecting horizontal edges and the other for detecting vertical edges. 
# The resulting gradient images can then be combined to estimate the magnitude and orientation of the edges
gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0) # Horizontal gradient
gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1) # Vertical gradient

In [ ]:
# Compute the surface normal using the image gradient
# The surface normal is a vector that is perpendicular to the surface at each point
# This code computes the surface normal at each pixel using the image gradient
nx = gx / np.sqrt(gx**2 + gy**2 + 1)
ny = gy / np.sqrt(gx**2 + gy**2 + 1)
nz = -1 / np.sqrt(gx**2 + gy**2 + 1)

In [ ]:
# Integrate the surface normals to estimate the surface geometry
# Integrates the x, y, and z components of the surface normal to estimate the 3D coordinates of the surface
z = cv2.integral(nz)[1:,1:]
y = cv2.integral(ny*z)[1:,1:]
x = cv2.integral(nx*z)[1:,1:]

In [ ]:
# Import libraries to view the surface geometry
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Visualize the surface geometry estimated from the surface normals using a 3D surface plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, z, cmap='jet')
plt.show()